In [ ]:
# Yash Thakur (B-75)

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Set seed for reproducibility
np.random.seed(42)

# Generate dummy features
n_samples = 200

Temperature = np.random.uniform(150, 200, n_samples)        # Temperature (°C)
FlowRate = np.random.uniform(50, 150, n_samples)             # Flow Rate (mL/min)
Pressure = np.random.uniform(1, 5, n_samples)                # Pressure (bar)
Humidity = np.random.uniform(20, 60, n_samples)              # Humidity (%)
InletAirTemp = np.random.uniform(170, 210, n_samples)        # Inlet Air Temperature (°C)

# Generate target output (Moisture Content %) with some dependency
# Assume Moisture Content inversely proportional to Temperature, FlowRate, etc.
MoistureContent = (
    25
    - 0.05 * Temperature
    - 0.03 * FlowRate
    + 0.2 * Pressure
    + 0.1 * Humidity
    - 0.04 * InletAirTemp
    + np.random.normal(0, 1, n_samples)  # Some noise
)

# Create DataFrame
data = pd.DataFrame({
    'Temperature': Temperature,
    'FlowRate': FlowRate,
    'Pressure': Pressure,
    'Humidity': Humidity,
    'InletAirTemp': InletAirTemp,
    'Target_MoistureContent': MoistureContent
})

# Save to CSV (Optional)
# data.to_csv('coconut_spray_data.csv', index=False)

# Split features and target
X = data[['Temperature', 'FlowRate', 'Pressure', 'Humidity', 'InletAirTemp']].values
y = data[['Target_MoistureContent']].values

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print("Dummy dataset created successfully!")


Dummy dataset created successfully!


In [2]:
# 2. Build Neural Network Model
def create_nn_model(input_dim, learning_rate=0.001, neurons=10):
    model = Sequential()
    model.add(Dense(neurons, input_dim=input_dim, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse')
    return model



In [4]:
! pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.5 MB/s eta 0:00:00


In [5]:
from deap import base, creator, tools, algorithms
import random

In [6]:
# 3. Define Genetic Algorithm Functions
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize MSE
creator.create("Individual", list, fitness=creator.FitnessMin)

# Genetic Algorithm settings
def evaluate(individual):
    learning_rate = individual[0]
    neurons = int(individual[1])

    model = create_nn_model(X_train.shape[1], learning_rate, neurons)
    model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)  # Train the model
    loss = model.evaluate(X_test, y_test, verbose=0)  # Evaluate model on test data
    return loss,  # Return as a tuple

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, 0.0001, 0.1)  # Learning rate range
toolbox.register("attr_neuron", random.randint, 5, 100)  # Neurons range
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_float, toolbox.attr_neuron), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

In [8]:
from keras.models import Sequential
from keras.layers import Dense

# 4. Run Genetic Algorithm
def run_ga_nn():
    pop_size = 10
    generations = 5
    population = toolbox.population(n=pop_size)

    # Run the evolutionary algorithm
    algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=generations, verbose=True)

    # Get the best individual
    best_individual = tools.selBest(population, 1)[0]
    print(f"Best parameters: Learning rate = {best_individual[0]}, Neurons = {int(best_individual[1])}")

    # Train final model using best parameters
    best_learning_rate = best_individual[0]
    best_neurons = int(best_individual[1])

    final_model = create_nn_model(X_train.shape[1], best_learning_rate, best_neurons)
    final_model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1)

    # Evaluate final model
    final_loss = final_model.evaluate(X_test, y_test)
    print(f"Final model MSE on test set: {final_loss}")

if __name__ == "__main__":
    run_ga_nn()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


gen	nevals
0  	10    
1  	6     
2  	8     
3  	5     
4  	7     
5  	10    
Best parameters: Learning rate = -0.0020103346046292, Neurons = 74
Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 583.0286 
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 122.6082 
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 33.5242 
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 39.7849 
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 17.4182 
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 12.4113 
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10.9166 
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.4194 
Epoch 9/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.7443 
Epoch 10/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.7332 
Epoch 11/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.4241 
Epoch 12/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.7725 
Epoch 13/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step -